# **PART 3**

In [1]:
import pandas as pd

df_test_cases = pd.read_csv('/content/pronoun_testcases.csv')
display(df_test_cases.head())

,input_text,target_gender,expected_output
0,He is going to the market.,female,She is going to the market.
1,His book is on the table.,female,Her book is on the table.
2,I saw him yesterday.,female,I saw her yesterday.
3,He hurt himself.,female,She hurt herself.
4,I called him last night.,female,I called her last night.


In [2]:
pronoun_map = {
    'male_to_female': {
        'he': 'she',
        'him': 'her',
        'his': 'her',
        'himself': 'herself'
    },
    'female_to_male': {
        'she': 'he',
        'her': 'him',
        'hers': 'his',
        'herself': 'himself'
    }
}

print(pronoun_map)

{'male_to_female': {'he': 'she', 'him': 'her', 'his': 'her', 'himself': 'herself'}, 'female_to_male': {'she': 'he', 'her': 'him', 'hers': 'his', 'herself': 'himself'}}


In [3]:
import re

def transform_pronouns(sentence, target_gender):
    words = sentence.split()
    transformed_words = []

    for word in words:
        # Separate punctuation from the word
        match = re.match(r"(\W*)(.*?)(\W*)$", word)
        leading_punc, core_word, trailing_punc = match.groups()

        lower_word = core_word.lower()
        transformed_core_word = core_word

        if target_gender == 'female' and lower_word in pronoun_map['male_to_female']:
            transformed_core_word = pronoun_map['male_to_female'][lower_word]
            # Preserve original casing if possible
            if core_word[0].isupper():
                transformed_core_word = transformed_core_word.capitalize()
        elif target_gender == 'male' and lower_word in pronoun_map['female_to_male']:
            transformed_core_word = pronoun_map['female_to_male'][lower_word]
            # Preserve original casing if possible
            if core_word[0].isupper():
                transformed_core_word = transformed_core_word.capitalize()

        transformed_words.append(leading_punc + transformed_core_word + trailing_punc)

    return ' '.join(transformed_words)

# Test the function with an example
test_sentence = "He is going to the market."
test_target_gender = "female"
transformed_sentence = transform_pronouns(test_sentence, test_target_gender)
print(f"Original: {test_sentence}")
print(f"Transformed: {transformed_sentence}")

test_sentence_2 = "She hurt herself!"
test_target_gender_2 = "male"
transformed_sentence_2 = transform_pronouns(test_sentence_2, test_target_gender_2)
print(f"Original: {test_sentence_2}")
print(f"Transformed: {transformed_sentence_2}")

Original: He is going to the market.
Transformed: She is going to the market.
Original: She hurt herself!
Transformed: He hurt himself!


In [4]:
df_test_cases['transformed_output'] = df_test_cases.apply(
    lambda row: transform_pronouns(row['input_text'], row['target_gender']),
    axis=1
)
display(df_test_cases.head())

,input_text,target_gender,expected_output,transformed_output
0,He is going to the market.,female,She is going to the market.,She is going to the market.
1,His book is on the table.,female,Her book is on the table.,Her book is on the table.
2,I saw him yesterday.,female,I saw her yesterday.,I saw her yesterday.
3,He hurt himself.,female,She hurt herself.,She hurt herself.
4,I called him last night.,female,I called her last night.,I called her last night.


In [5]:
matches = (df_test_cases['transformed_output'] == df_test_cases['expected_output']).sum()
total_cases = len(df_test_cases)
accuracy = matches / total_cases
print(f"Accuracy of the pronoun transformation system: {accuracy:.2f}")

Accuracy of the pronoun transformation system: 0.81
